In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

from inference_toolbox import Parameter, Model, Likelihood, Sampler, Visualiser
from normaliser import Normaliser
from box_gridder import BoxGridder
from gaussian_processor import GaussianProcessor


import numpy as np

In [2]:
# Import and select data.
all_data = pd.read_csv('total_data.csv')

# Import and select metadata.
metadata = pd.read_csv('data_summary.csv',
    usecols = ['Experiment', 'Wind_Dir', 'WindSpeed', 'boat.lat', 'boat.lon']
)

In [3]:

normaliser = Normaliser(all_data, metadata)

all_experiments = normaliser.get_experiments_list()

selected_experiments = np.delete(all_experiments, np.where(all_experiments == 'Control'))

normalised_data = normaliser.normalise_data(selected_experiments)


In [4]:
box_gridder = BoxGridder(normalised_data)

averaged_df = box_gridder.get_averages([200,200,50],False)
# box_gridder.visualise_average_data(averaged_df)

# averaged_df

c:\Users\Sam\OneDrive\Documents\PhD\Gaussian_plume_models\plume_inference\box_gridder.py:89: RuntimeWarning: invalid value encountered in true_divide
  averages = sums/counts


In [5]:
# gp = GaussianProcessor(averaged_df, kernel = 'matern_white', data_norm='sqrt')
# training_data, test_data = gp.train_test_split()
# print(training_data)
# gp.train_gp(training_data)
# gp.test(test_data)
# grid = box_gridder.get_grid([10,10,10])
# gp.predict_from_gp(grid, threeD=True, save=True, log_results=True)
# gp.animate() 

In [6]:
training_data, testing_data = train_test_split(averaged_df, test_size=0.2)

In [7]:
# Initialize parameter series
params = pd.Series({},dtype='float64')

# Parameter Assignment
I_y = Parameter(init_val = 0.1, step_select = "gamma", step_size = 0.1, prior_select = "no prior")
I_y.add_prior_param("mu",0.1)
I_y.add_prior_param("sigma",0.1)
params['I_y'] = I_y

I_z = Parameter(init_val = 0.1, step_select = "gamma", step_size = 0.1, prior_select = "no prior")
I_z.add_prior_param("mu",0.1)
I_z.add_prior_param("sigma",0.1)
params['I_z'] = I_z

Q = Parameter(init_val = 3e13, step_select = "gamma", step_size = 1e12, prior_select = "no prior")
Q.add_prior_param("mu",3e13)
Q.add_prior_param("sigma",1e12)
params['Q'] = Q

# Model Assignment
model = Model('GPM_alt_norm')
model.add_model_param("H",10)

# Likelihood function assigmnent
likelihood = Likelihood("gaussian_fixed_sigma")
likelihood.add_likelihood_param("sigma",1e10)
# likelihood.add_likelihood_param("lambda_1",1)
# likelihood.add_likelihood_param("lambda_2",0.05)

# Initialize the sampler
sampler = Sampler(params, model, likelihood, training_data)
hyperparams = sampler.get_hyperparams()

# Sample the parameters
params_samples, acceptance_rate = sampler.sample_all(100000)

params_mean = sampler.get_mean_samples(params_samples)
params_lower = sampler.get_lower_samples(params_samples)
params_upper = sampler.get_upper_samples(params_samples)

Running sample 1000...
Running sample 2000...
Running sample 3000...
Running sample 4000...
Running sample 5000...
Running sample 6000...
Running sample 7000...
Running sample 8000...
Running sample 9000...
Running sample 10000...
Running sample 11000...
Running sample 12000...
Running sample 13000...
Running sample 14000...
Running sample 15000...
Running sample 16000...
Running sample 17000...
Running sample 18000...
Running sample 19000...
Running sample 20000...
Running sample 21000...
Running sample 22000...
Running sample 23000...
Running sample 24000...
Running sample 25000...
Running sample 26000...
Running sample 27000...
Running sample 28000...
Running sample 29000...
Running sample 30000...
Running sample 31000...
Running sample 32000...
Running sample 33000...
Running sample 34000...
Running sample 35000...
Running sample 36000...
Running sample 37000...
Running sample 38000...
Running sample 39000...
Running sample 40000...
Running sample 41000...
Running sample 42000...
R

c:\Users\Sam\OneDrive\Documents\PhD\Gaussian_plume_models\plume_inference\inference_toolbox.py:35: RuntimeWarning: divide by zero encountered in double_scalars
  return stats.gamma.logpdf(val, a, scale=1/beta)
c:\Users\Sam\OneDrive\Documents\PhD\Gaussian_plume_models\plume_inference\inference_toolbox.py:35: RuntimeWarning: overflow encountered in double_scalars
  return stats.gamma.logpdf(val, a, scale=1/beta)


Running sample 50000...
Running sample 51000...
Running sample 52000...
Running sample 53000...
Running sample 54000...
Running sample 55000...
Running sample 56000...
Running sample 57000...
Running sample 58000...
Running sample 59000...
Running sample 60000...
Running sample 61000...
Running sample 62000...
Running sample 63000...
Running sample 64000...
Running sample 65000...
Running sample 66000...
Running sample 67000...
Running sample 68000...
Running sample 69000...
Running sample 70000...
Running sample 71000...
Running sample 72000...
Running sample 73000...
Running sample 74000...
Running sample 75000...
Running sample 76000...
Running sample 77000...
Running sample 78000...
Running sample 79000...
Running sample 80000...
Running sample 81000...
Running sample 82000...
Running sample 83000...
Running sample 84000...
Running sample 85000...
Running sample 86000...
Running sample 87000...
Running sample 88000...
Running sample 89000...
Running sample 90000...
Running sample 9

In [8]:
x = np.linspace(0.1, 1200, 81)
y = np.linspace(-1000, 1000, 81)
z = np.linspace(0,200,81)

domain = np.array([x,y,z]).T

params_all = [params_lower, params_mean, params_upper]

visualiser = Visualiser(testing_data, params_all, model, hyperparams, acceptance_rate)
instance = visualiser.visualise_results(domain, save = True)
visualiser.get_traceplot(params_samples, instance, save = True)
visualiser.animate(instance)